# Import elk snapshot in another fabric slice

## Import

In [ ]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")
import mflib
print(f"MFLib version  {mflib.__version__} " )
from mflib.mflib import MFLib

In [ ]:
import importlib
import importlib
from mflib.mf_data_transfer import mf_data_export

## Slice Info

In [ ]:
slice_name = "backup-testing"
node1_name = 'data-backup'

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

try:
    node = slice.get_node(name=node1_name)
except Exception as e:
    print(f"Fail: {e}")

## Start elk container to use the snapshot

### upload elasticsearch.yml to the node

In [ ]:
# Upload an example elasticsearch.yml file
node.upload_file(local_file_path="/home/fabric/work/jupyter-examples/fabric_examples/mflib/Pinyi/data_transfer/elk_docker/elasticsearch.yml", 
                 remote_file_path='/home/ubuntu/elasticsearch.yml')
cmd = "sudo chown root:root /home/ubuntu/elasticsearch.yml"
stdout, stderr=node.execute(cmd)

### create docker volumes

In [ ]:
cmd = "sudo docker volume create elk_snapshotbackup"
stdout, stderr=node.execute(cmd)
cmd = "sudo docker volume create elk_data01"
stdout, stderr=node.execute(cmd)

### Start the container

In [ ]:
cmd = f'''
        sudo docker run -dit --name elasticsearch \
        --user 0:0 \
        -p 9200:9200 \
        -e "discovery.type=single-node" \
        -e "path.repo=/usr/share/elasticsearch/backup" \
        -v /home/ubuntu/elasticsearch.yml:/usr/share/elasticsearch/config/elasticsearch.yml \
        -v elk_data01:/usr/share/elasticsearch/data \
        -v elk_snapshotbackup:/usr/share/elasticsearch/backup \
        --ulimit memlock=-1:-1 \
        --log-driver json-file \
        elasticsearch:7.13.0
      '''
stdout, stderr=node.execute(cmd)

In [ ]:
node.execute("sudo docker ps")

### Change snapshot dir permission

In [ ]:
cmd = f"sudo chown -R 1000:1000 /var/lib/docker/volumes/elk_snapshotbackup/_data"
stdout, stderr=node.execute(cmd)

### Register a backup repository

In [ ]:
repo_name = "backup_repository"
default_dir = "/usr/share/elasticsearch/backup"
cmd = f'curl -X PUT "http://localhost:9200/_snapshot/{repo_name}?pretty" -H "Content-Type: application/json" -d \'{{ "type": "fs", "settings": {{ "location": "{default_dir}" }} }}\''
stdout, stderr=node.execute(cmd)

### Create a snapshot

In [ ]:
snapshot_name = "snapshot_test"
json_str = '"ignore_unavailable": true, "include_global_state": false'
cmd = f'curl -X PUT "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?wait_for_completion=true&pretty" -H "Content-Type: application/json" -d \'{json_str}\''
stdout, stderr= node.execute(cmd)

In [ ]:
# Upload an example elasticsearch.yml file
node.upload_file(local_file_path="/home/fabric/work/data-transfer-jupyter/jupyter-examples/fabric_examples/mflib/data_transfer_service/snapshots/elk_backup_0529.tar", 
                 remote_file_path='/home/ubuntu/elk_backup_0529.tar')

### Replace the file with the tar file

In [ ]:
snapshot_dir = "/var/lib/docker/volumes/elk_snapshotbackup/_data"
cmd = f"sudo rm -rf {snapshot_dir}"
stdout, stderr= node.execute(cmd)

In [ ]:
# untar backup snapshot files to snapshot dir
snapshot_parent_dir = "/var/lib/docker/volumes/elk_snapshotbackup/"
elk_tar_file_name = "elk_backup_0529.tar"
elk_tar_path = "/home/ubuntu/elk/download"
cmd = f'sudo tar -xvf {elk_tar_path}/{elk_tar_file_name} -C {snapshot_parent_dir}'
node.execute(cmd, quiet=True)

In [ ]:
cmd = f"sudo chown -R 1000:1000 /var/lib/docker/volumes/elk_snapshotbackup/_data"
stdout, stderr=node.execute(cmd)
cmd = f"sudo ls {snapshot_parent_dir}/_data"
node.execute(cmd, quiet=True)

### Restart the container

In [ ]:
cmd = "sudo docker restart elasticsearch"
stdout, stderr= node.execute(cmd)

### Check snapshot

In [ ]:
# If you dont see the snapshot, you need to restart the container again(above step)
repo_name="backup_repository"
snapshot_name = "snapshot_test"
cmd = f"curl -X GET 'http://localhost:9200/_cat/snapshots/{repo_name}?v'"
stdout, stderr= node.execute(cmd)

In [ ]:
cmd = f'curl -X GET "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}?pretty"'
stdout, stderr= node.execute(cmd)

### Delete indice if any

In [ ]:
cmd = f'curl -X DELETE "http://localhost:9200/_all"'
stdout, stderr= node.execute(cmd)

### Check indice

In [ ]:
cmd= f'curl -XGET "http://localhost:9200/_cat/indices?pretty"'
stdout, stderr= node.execute(cmd)

### Restore using the snapshot

In [ ]:
# Need to specify indice names to restore
indice_str= f'"indices": "filebeat-7.13.2-2023.05.29-000001,metricbeat-7.13.2-2023.05.29-000001,packetbeat-7.13.2-2023.05.29-000001",'
repo_name="backup_repository"
snapshot_name = "snapshot_test"
cmd = f'curl -X POST "http://localhost:9200/_snapshot/{repo_name}/{snapshot_name}/_restore?pretty" -H "Content-Type: application/json" -d \'{{{indice_str} "index_settings": {{ "index.number_of_replicas": 1 }} }}\''
#print (cmd)
stdout, stderr= node.execute(cmd)

### Check progress

In [ ]:
cmd = f'curl -XGET "http://localhost:9200/_cat/recovery?v"'
stdout, stderr= node.execute(cmd)

### After the process finishes, check the indice

In [ ]:
# You should see the same indice as what you exported 
cmd= f'curl -XGET "http://localhost:9200/_cat/indices?pretty"'
stdout, stderr= node.execute(cmd)

## Set up kibana

In [ ]:
cmd = f'sudo docker run -d --name kibana --link elasticsearch:elasticsearch -p 5601:5601 docker.elastic.co/kibana/kibana:7.13.0'
stdout, stderr= node.execute(cmd)

## View data in the web ui

##### Run 'ssh -L 10030:localhost:5601 -F ssh_config -i slice_key ubuntu@{node_ip}' ; Then go to http://localhost:10030